In [ ]:
import os
import copy
import numpy as np

from itertools import product

Sample some random seeds for train splits.

In [ ]:
np.random.randint(0x7fff_ffff, size=(13,))

Dataset splits

In [ ]:
from cplxpaper.mnist import dataset as mnist_like

datasets = {
    "mnist": (
        1641730374,
        str(mnist_like.MNIST_Train),
        str(mnist_like.MNIST_Test),
    ),
#     "kmnist": (
#         102048205,
#         str(mnist_like.KMNIST_Train), 
#         str(mnist_like.KMNIST_Test),
#     ),
#     "fashion-mnist": (
#         1526761432,
#         str(mnist_like.FashionMNIST_Train), 
#         str(mnist_like.FashionMNIST_Test),
#     ),
#     "emnist": (
#         605446338,
#         str(mnist_like.EMNIST_Letters_Train), 
#         str(mnist_like.EMNIST_Letters_Test),
#     ),
}

$\mathbb{R}$-Model stages

In [ ]:
from cplxpaper.mnist.models import real

real_model_stages = {
    "twolayerdense": {
        "model__cls": [str(real.TwoLayerDenseModel),],
        "stages__sparsify__model__cls": [str(real.TwoLayerDenseModelARD),],
        "stages__fine-tune__model__cls": [str(real.TwoLayerDenseModelMasked),],
    },
    "simpledense": {
        "model__cls": [str(real.SimpleDenseModel),],
        "stages__sparsify__model__cls": [str(real.SimpleDenseModelARD),],
        "stages__fine-tune__model__cls": [str(real.SimpleDenseModelMasked),],
    },
    "simpleconv": {
        "model__cls": [str(real.SimpleConvModel),],
        "stages__sparsify__model__cls": [str(real.SimpleConvModelARD),],
        "stages__fine-tune__model__cls": [str(real.SimpleConvModelMasked),],
    },
}

$\mathbb{C}$-model stages

In [ ]:
import cplxpaper.mnist.models.complex as cplx

cplx_model_stages = {
    "twolayerdense": {
        "model__cls": [str(cplx.TwoLayerDenseModel),],
        "stages__sparsify__model__cls": [str(cplx.TwoLayerDenseModelARD),],
        "stages__fine-tune__model__cls": [str(cplx.TwoLayerDenseModelMasked),],
    },
    "simpledense": {
        "model__cls": [str(cplx.SimpleDenseModel),],
        "stages__sparsify__model__cls": [str(cplx.SimpleDenseModelARD),],
        "stages__fine-tune__model__cls": [str(cplx.SimpleDenseModelMasked),],
    },
    "simpleconv": {
        "model__cls": [str(cplx.SimpleConvModel),],
        "stages__sparsify__model__cls": [str(cplx.SimpleConvModelARD),],
        "stages__fine-tune__model__cls": [str(cplx.SimpleConvModelMasked),],
    },
}

Load default config

In [ ]:
import json
from pkg_resources import resource_stream

with resource_stream("cplxpaper.mnist", "template.json") as fin:
    options = json.load(fin)

Reset roots

In [ ]:
from cplxpaper.auto.parameter_grid import get_params, set_params, special_params

experiment_folder = os.path.expanduser("~/Github/complex_paper/experiments/mnist/data")

options = set_params(options, **{
    "datasets": {
        "train": {"root": experiment_folder,},
        "test": {"root": experiment_folder,},
    }, 
    "model": {},
    "stages__dense__model": {},
    "stages__sparsify__model": {},
    "stages__fine-tune__model": {},
})

Construct a grid of kl-divergence coefficients (for size-averaged kl-div terms)

In [ ]:
kl_divs = np.logspace(-6, 0, num=19)

```python
kl_divs = np.sort(np.r_[
    1 * np.logspace(-6, -1, 6),
    5 * np.logspace(-6, -2, 5),
    1 * np.linspace(0.2, 1, 9),
    0.95
])
```

Define default grid parameters for mnist-like experiments.

In [ ]:
base_grid = {
    "n_copy": [*range(5)],
    "datasets__train__train_size": [10000],

    "objective_terms__kl_div__reduction": ["mean"],  # size averaged kl-div -- more unifor sparsification (poor)

#     "objective_terms__kl_div__reduction": ["sum"],  # sum of kl-div divided by n_samples
#     "objective_terms__kl_div__coef": [1e-4],

    "feeds__train__dataset": ["train"],
    "feeds__test__dataset": ["test"],
    "feeds__train__batch_size": [128],
    
    # train 50-100-75, fine-tune with restart, and grad-clip l2 norm 0.5
    "stages__dense__n_epochs": [50],
    "stages__dense__grad_clip": [0.5],
    "stages__sparsify__n_epochs": [100],
    "stages__sparsify__restart": [False],
    "stages__sparsify__reset": [False],
    "stages__sparsify__grad_clip": [0.5],
    "stages__fine-tune__n_epochs": [75],
    "stages__fine-tune__restart": [True],
    "stages__fine-tune__reset": [True],
    "stages__fine-tune__grad_clip": [0.5],

    # kl-div weights come from a moderately grid
    "stages__sparsify__objective__kl_div": kl_divs,
}

<br>

## Grid 1: MNIST-like datasets,  Real Models

In [ ]:
grid_real_mnistlike = []
for model, dataset in product(real_model_stages, datasets):
    grid_real = copy.deepcopy(base_grid)
    if dataset == "emnist":
        grid_real.update({"model__n_outputs": [26],})
    else:
        grid_real.update({"model__n_outputs": [10],})

    random_state, train__cls, test__cls = datasets[dataset]
    grid_real.update(real_model_stages[model])
    grid_real.update({
        "datasets__train__random_state": [random_state],
        "datasets__train__cls": [train__cls,],
        "datasets__test__cls": [test__cls,],
        "features": [{"cls": "<class 'cplxpaper.auto.feeds.FeedRawFeatures'>"}],
    })

    grid_real_mnistlike.append(grid_real)

<br>

## Grid 2: MNIST-like datasets,  Complex Models

In [ ]:
grid_cplx_mnistlike = []
for model, dataset in product(cplx_model_stages, datasets):
    grid_cplx = copy.deepcopy(base_grid)
    if dataset == "emnist":
        grid_cplx.update({"model__n_outputs": [26],})
    else:
        grid_cplx.update({"model__n_outputs": [10],})

    random_state, train__cls, test__cls = datasets[dataset]
    grid_cplx.update(cplx_model_stages[model])
    grid_cplx.update({
        "datasets__train__random_state": [random_state],
        "datasets__train__cls": [train__cls,],
        "datasets__test__cls": [test__cls,],
        "features": [{
            "cls": "<class 'cplxpaper.auto.feeds.FeedFourierFeatures'>",
            "signal_ndim": 2,
            "shift": False,
            "cplx": True,
        }],
    })

    grid_cplx_mnistlike.append(grid_cplx)

<br>

In [ ]:
from sklearn.model_selection import ParameterGrid

grid = ParameterGrid([
    *grid_real_mnistlike,
    *grid_cplx_mnistlike
])

folder = os.path.join(".", "grids", "minst-like")
os.makedirs(folder)

In [ ]:
import tqdm
from cplxpaper.auto.utils import get_class

for i, par in enumerate(tqdm.tqdm(grid)):
    par, special = special_params(**par)
    assert not special

    local = set_params(copy.deepcopy(options), **par, device=None)
    n_copy = local.pop("n_copy")

    # format the name
    dataset = get_class(local["datasets"]["test"]["cls"]).__name__
    dataset = dataset.rsplit("_", 1)[0]
    kind = "cplx" if local["features"].get("signal_ndim", 1) == 2 else "real"
    model = get_class(local["model"]["cls"]).__name__
    
    manifest = f"{dataset}-{kind}-{model}-{n_copy} {i:04d}.json"
    filename = os.path.join(folder, manifest)
    json.dump(local, open(filename, "w"))

<br>